In [ ]:
pip install pandas xgboost catboost lightgbm tabPFN

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import RidgeCV, LassoCV
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor

declaring the dataframe and split the data to matrix X and vector Y

A dataset for classification task the output is the income column.
5000 rows, 12 cols 7 numerical features.
 you can find the dataset here: https://www.kaggle.com/datasets/ajinilpatel/energy-consumption-prediction

In [ ]:
df_reg = pd.read_csv('Energy_consumption_dataset.csv')
df_reg

In [ ]:
df_reg.dtypes

### 8:
Creating a preprocessing function:
* I decided to create sub functions first and call them from a 'main' function.

A:  CHECKING FOR MISSING VALUES (THERE ISN'T MISSING VALUES) AND CATEGORICAL COLS

In [ ]:
df_reg.isnull().sum()

In [ ]:
def handle_missing_values(df):
    print("Checking for missing values:")
    print(df.isnull().sum())
    # df = df.fillna()  THERE IS NO NULL VALUES ANYWAY SO WE DON'T NEED THAT (:
    return df

def remove_high_cardinality_columns(df, max_unique=4):
    cat_cols = df.select_dtypes(include=['object', 'category','string']).columns
    removed_cols = []

    for col in cat_cols:
        if df[col].nunique() > max_unique:
            df.drop(columns=[col], inplace=True)
            removed_cols.append(col)

    print(f"Removed high-cardinality columns (>{max_unique} unique values): {removed_cols}")
    return df

def encode_categorical_variables(df):
    cat_cols = df.select_dtypes(include=['object', 'category']).columns
    df_encoded = pd.get_dummies(df, columns=cat_cols, drop_first=True)
    print(f"Encoded categorical columns: {list(cat_cols)}")
    return df_encoded


B: SPLITTING FOR TRAIN/TEST:

In [ ]:
def split_data_reg(X, y, test_size=0.2, random_state=42):
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state
    )
    print(f"Training set: {X_train.shape}, Test set: {X_test.shape}")
    return X_train, X_test, y_train, y_test


C: STANDARDIZE THE DATA:

In [ ]:
def standardize_features_reg(X_train, X_test):
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    print("Standardization complete.")
    return X_train_scaled, X_test_scaled


In [ ]:
def preprocess_regression_data(df, target_column):
    X_reg = df.drop(columns=[target_column])
    y_reg = df[target_column]
    
    X_reg = handle_missing_values(X_reg)
    X_reg = remove_high_cardinality_columns(X_reg)
    X_reg = encode_categorical_variables(X_reg)

    X_train_reg, X_test_reg, y_train_reg, y_test_reg = split_data_reg(X_reg, y_reg)

    X_train_scaled_reg, X_test_scaled_reg = standardize_features_reg(X_train_reg, X_test_reg)

    return X_train_scaled_reg, X_test_scaled_reg, y_train_reg, y_test_reg


In [ ]:
X_train_scaled_reg, X_test_scaled_reg, y_train_reg, y_test_reg = preprocess_regression_data(df_reg, 'EnergyConsumption')

### 9:
Train a linear regression model:

In [ ]:
lr_model = LinearRegression()

lr_model.fit(X_train_scaled_reg, y_train_reg)

### 10: 
Making Predictions and evaluating the model:

In [ ]:
# Custom implementation of MAPE 
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100


In [ ]:
# Predictions on test set and train set
y_train_pred_lr = lr_model.predict(X_train_scaled_reg)
y_test_pred_lr = lr_model.predict(X_test_scaled_reg)

# Metrics:
r2_train = r2_score(y_train_reg, y_train_pred_lr)
rmse_train = mean_squared_error(y_train_reg, y_train_pred_lr)
mae_train = mean_absolute_error(y_train_reg, y_train_pred_lr)
mape_train = mean_absolute_percentage_error(y_train_reg, y_train_pred_lr)

r2_test = r2_score(y_test_reg, y_test_pred_lr)
rmse_test = mean_squared_error(y_test_reg, y_test_pred_lr)
mae_test = mean_absolute_error(y_test_reg, y_test_pred_lr)
mape_test = mean_absolute_percentage_error(y_test_reg, y_test_pred_lr)

print("Linear Regression Evaluation Metrics:\n")

print("Training Set:")
print(f"R^2: {r2_train:.4f}")
print(f"RMSE: {rmse_train:.4f}")
print(f"MAE: {mae_train:.4f}")
print(f"MAPE: {mape_train:.2f}%\n")

print("Test Set:")
print(f"R^2: {r2_test:.4f}")
print(f"RMSE: {rmse_test:.4f}")
print(f"MAE: {mae_test:.4f}")
print(f"MAPE: {mape_test:.2f}%")


### 11:
Polynomial Regression

Transforms features into polynomial features of the given degree.
Returns a new DataFrame X_poly

In [ ]:
def create_polynomial_features(X, degree):
    poly = PolynomialFeatures(degree=degree, include_bias=False)
    X_poly = poly.fit_transform(X)
    print(f"Polynomial features created for degree {degree}. Shape: {X_poly.shape}")
    return X_poly


Splitting for train/test and train models by Ridge and Lasso

In [ ]:
def evaluate_polynomial_models(X, y, degrees=[2, 3, 4]):
    results = []

    for d in degrees:
        print(f"\nEvaluating Polynomial Degree: {d}")

        # Create polynomial features
        X_poly = create_polynomial_features(X, d)

        # Split and scale
        X_train, X_test, y_train, y_test = train_test_split(X_poly, y, test_size=0.2, random_state=42)
        X_train_scaled, X_test_scaled = standardize_features_reg(X_train, X_test)

        # Ridge Regression with cross-validation
        ridge_model = RidgeCV(alphas=[0.01, 0.1, 1, 10, 100], cv=5)
        ridge_model.fit(X_train_scaled, y_train)
        y_pred_ridge = ridge_model.predict(X_test_scaled)
        r2_ridge = r2_score(y_test, y_pred_ridge)

        # Lasso Regression with cross-validation
        lasso_model = LassoCV(alphas=[0.01, 0.1, 1, 10, 100], cv=5, max_iter=20000)
        lasso_model.fit(X_train_scaled, y_train)
        y_pred_lasso = lasso_model.predict(X_test_scaled)
        r2_lasso = r2_score(y_test, y_pred_lasso)

        results.append({
            'degree': d,
            'ridge_alpha': ridge_model.alpha_,
            'lasso_alpha': lasso_model.alpha_,
            'ridge_r2': r2_ridge,
            'lasso_r2': r2_lasso,
            'best_model': 'Ridge' if r2_ridge > r2_lasso else 'Lasso',
            'best_r2': max(r2_ridge, r2_lasso)
        })

    return results


Run the models and retrieve the best model:

In [ ]:
results = evaluate_polynomial_models(X_train_scaled_reg, y_train_reg)

print("\nSummary of Polynomial Models:")
for res in results:
    print(f"Degree {res['degree']}:")
    print(f"  Ridge - Alpha: {res['ridge_alpha']} | R²: {res['ridge_r2']:.4f}")
    print(f"  Lasso - Alpha: {res['lasso_alpha']} | R²: {res['lasso_r2']:.4f}")
    print(f"  Best Model: {res['best_model']} with R²: {res['best_r2']:.4f}\n")

best = max(results, key=lambda x: x['best_r2'])
print("🏆 Best Overall Model:")
print(f"Degree: {best['degree']}")
print(f"Model: {best['best_model']}")
print(f"R²: {best['best_r2']:.4f}")
print(f"Alpha: {best['ridge_alpha'] if best['best_model']=='Ridge' else best['lasso_alpha']}")


🏆 Best Overall Model:
Degree: 2
Model: Lasso
R²: 0.3340
Alpha: 0.1

### 12:
What can be the problem with creating variables in degree 2+ and how can you solve it?
* the main problem is that the features grows significantly which can lead to overfitting that reduce the model's ability to generalize patterns.
* We can solve it we use regularization methods such as Lasso and Ridge which we'll explain about them in the next question.

What are Lasso and Ridge, when to use and what's the main difference between them?
* Lasso and ridge are penalize large coefficients:
* Use Lasso when you believe that some features are irrelevant, it shrinks those coefficients to zero and acting as a form of automatic **feature selection**
* Use Ridge when you believe that all features contribute to the outcome, but you want to prevent any single feature from dominating. Ridge shrinks all coefficients without removing them.
* Main Differences: 
* Lasso can eliminate features by setting their coefficients to zero (L1 regularization)
* Ridge reduces the magnitude of all coefficients (L2 regularization)

### 13:
Making prediction on train set and test set for the best model from 11 and display the metrics:

🏆 Best Overall Model:
Degree: 2
Model: Lasso
R²: 0.3340
Alpha: 0.1

In [ ]:
# Recreate polynomial features for degree 2
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly.fit_transform(X_train_scaled_reg)

# Split and scale again
X_train_poly, X_test_poly, y_train_poly, y_test_poly = train_test_split(
    X_poly, y_train_reg, test_size=0.2, random_state=42
)

X_train_poly_scaled, X_test_poly_scaled = standardize_features_reg(X_train_poly, X_test_poly)


In [ ]:
best_lasso = LassoCV(alphas=[0.1], cv=5, max_iter=20000)
best_lasso.fit(X_train_poly_scaled, y_train_poly)

y_train_pred_best_lasso = best_lasso.predict(X_train_poly_scaled)
y_test_pred_best_lasso = best_lasso.predict(X_test_poly_scaled)


In [ ]:
def evaluate_model(y_true, y_pred, label="Set"):
    r2 = r2_score(y_true, y_pred)
    rmse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred)
    print(f"{label} Evaluation:")
    print(f"R^2: {r2:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"MAE: {mae:.4f}")
    print(f"MAPE: {mape:.2f}%\n")


In [ ]:
# Evaluate on both sets
evaluate_model(y_train_poly, y_train_pred_best_lasso, "Training Set")
evaluate_model(y_test_poly, y_test_pred_best_lasso, "Test Set")


### 14: 
Train models with gridsearchcv on train set from 8

In [ ]:
def train_model_with_gridsearch(model, param_grid, X_train, y_train):
    grid = GridSearchCV(estimator=model, param_grid=param_grid, 
                        cv=5, scoring='r2', n_jobs=-1)
    grid.fit(X_train, y_train)
    return grid.best_estimator_, grid.best_params_

In [ ]:
XGB_model = XGBRegressor()
param_grid_XGB = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5]
}

best_XGBRegressor, best_XGB_params = train_model_with_gridsearch(
    XGB_model, param_grid_XGB, X_train_scaled_reg, y_train_reg
)
print("Best XGBRegressor params:", best_XGB_params)


In [ ]:
CatBoost_model = CatBoostRegressor(verbose=0)
param_grid_CatBoost = {
    'depth': [4, 6],
    'learning_rate': [0.01, 0.1]
}

best_CatBoostRegressor, best_CatBoost_params = train_model_with_gridsearch(
    CatBoost_model, param_grid_CatBoost, X_train_scaled_reg, y_train_reg
)
print("Best CatBoostRegressor params:", best_CatBoost_params)

In [ ]:
LGBM_model = LGBMRegressor()
param_grid_LGBM = {
    'num_leaves': [31, 64],
    'learning_rate': [0.01, 0.1]
}

best_LGBMRegressor, best_LGBM_params = train_model_with_gridsearch(
    LGBM_model, param_grid_LGBM, X_train_scaled_reg, y_train_reg
)
print("Best LGBMRegressor params:", best_LGBM_params)

In [ ]:
RF_model = RandomForestRegressor()
param_grid_RF = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10]
}

best_RFRegressor, best_RF_params = train_model_with_gridsearch(
    RF_model, param_grid_RF, X_train_scaled_reg, y_train_reg
)
print(best_RF_params)

In [ ]:
from tabpfn import TabPFNRegressor
import torch


# Initialize the model
tabpfn_model = TabPFNRegressor(device='cuda' if torch.cuda.is_available() else 'cpu', ignore_pretraining_limits=True)

# Fit on the training set (already scaled from step 8)
tabpfn_model.fit(X_train_scaled_reg, y_train_reg)

### 15:
Making predictions on train set and test set for each of the models and displaying the metrics:

In [ ]:
def print_metrics(y_train, y_train_pred, y_test, y_test_pred, model_name):
    print(f"\n{model_name} Evaluation")

    print("Training Set:")
    print(f"R^2:   {r2_score(y_train, y_train_pred):.4f}")
    print(f"RMSE: {mean_squared_error(y_train, y_train_pred):.4f}")
    print(f"MAE:  {mean_absolute_error(y_train, y_train_pred):.4f}")
    print(f"MAPE: {mean_absolute_percentage_error(y_train, y_train_pred):.2f}%")

    print("\nTest Set:")
    print(f"R^2:   {r2_score(y_test, y_test_pred):.4f}")
    print(f"RMSE: {mean_squared_error(y_test, y_test_pred):.4f}")
    print(f"MAE:  {mean_absolute_error(y_test, y_test_pred):.4f}")
    print(f"MAPE: {mean_absolute_percentage_error(y_test, y_test_pred):.2f}%")


In [ ]:
y_train_pred_xgb = best_XGBRegressor.predict(X_train_scaled_reg)
y_test_pred_xgb = best_XGBRegressor.predict(X_test_scaled_reg)

print_metrics(y_train_reg, y_train_pred_xgb, y_test_reg, y_test_pred_xgb, "XGBoost")

In [ ]:
y_train_pred_cat = best_CatBoostRegressor.predict(X_train_scaled_reg)
y_test_pred_cat = best_CatBoostRegressor.predict(X_test_scaled_reg)

print_metrics(y_train_reg, y_train_pred_cat, y_test_reg, y_test_pred_cat, "CatBoost")

In [ ]:
y_train_pred_lgbm = best_LGBMRegressor.predict(X_train_scaled_reg)
y_test_pred_lgbm = best_LGBMRegressor.predict(X_test_scaled_reg)

print_metrics(y_train_reg, y_train_pred_lgbm, y_test_reg, y_test_pred_lgbm, "LGBM")

In [ ]:
y_train_pred_rf = best_RFRegressor.predict(X_train_scaled_reg)
y_test_pred_rf = best_RFRegressor.predict(X_test_scaled_reg)

print_metrics(y_train_reg, y_train_pred_rf, y_test_reg, y_test_pred_rf, "Random Forest")

Due to computational limitations, TabPFN predicted on a 1000-sample subset. While this doesn't allow a fully fair comparison, it offers insight into TabPFN's performance characteristics on limited data.

In [ ]:
X_train_small_reg = X_train_scaled_reg[:1000]
y_train_small_reg = y_train_reg[:1000]

X_test_small_reg = X_test_scaled_reg[:1000]
y_test_small_reg = y_test_reg[:1000]

y_train_tabpfn = tabpfn_model.predict(X_train_small_reg)
y_test_tabpfn = tabpfn_model.predict(X_test_small_reg)
print_metrics(y_train_small_reg, y_train_tabpfn, y_test_small_reg, y_test_tabpfn, "tabPFN")

But I ran the predictions in VS code on my local GPU on the original data print_metrics(y_train_reg, y_train_pred_tabpfn, y_test_reg, y_test_pred_tabpfn, "tabPFN") and I markdown the results to be consistent with answer with my answer in 17.a

In [ ]:
# RUN IT WITH GPU OTHERWISE IT STUCK
'''
y_train_pred_tabpfn = tabpfn_model.predict(X_train_scaled_reg)
y_test_pred_tabpfn = tabpfn_model.predict(X_test_scaled_reg)
print_metrics(y_train_reg, y_train_pred_tabpfn, y_test_reg, y_test_pred_tabpfn, "tabPFN")
'''

### 16:
Creating summary table with the 5 models:

In [ ]:
summary = {
    'Model': ['XGBoost', 'CatBoost', 'LGBM', 'Random Forest', 'tabPFN','LinearRegression', 'Best Lasso'],
    'RMSE Train': [
        mean_squared_error(y_train_reg, y_train_pred_xgb),
        mean_squared_error(y_train_reg, y_train_pred_cat),
        mean_squared_error(y_train_reg, y_train_pred_lgbm),
        mean_squared_error(y_train_reg, y_train_pred_rf),
        mean_squared_error(y_train_small_reg, y_train_tabpfn),
        mean_squared_error(y_train_reg, y_train_pred_lr),
        mean_squared_error(y_train_poly, y_train_pred_best_lasso)
    ],
    
    'RMSE Test': [
        mean_squared_error(y_test_reg, y_test_pred_xgb),
        mean_squared_error(y_test_reg, y_test_pred_cat),
        mean_squared_error(y_test_reg, y_test_pred_lgbm),
        mean_squared_error(y_test_reg, y_test_pred_rf),
        mean_squared_error(y_test_reg, y_test_tabpfn),
        mean_squared_error(y_test_reg, y_test_pred_lr),
        mean_squared_error(y_test_poly, y_test_pred_best_lasso)
    ]
}

df_summary = pd.DataFrame(summary).round(4)
print(df_summary)


Marking down the table because i wanted to be consistent with my answer in 17.a


| Model             | RMSE Train | RMSE Test |
|-------------------|------------|-----------|
| XGBOOST           | 40.8120    | 64.8693   |
| CatBoost          | 52.1670    | 60.0235   |
| LGBM              | 29.1277    | 63.2355   |
| Random Forest     | 25.6229    | 62.6501   |
| tabPFN            | 54.0252    | 59.8595   |
| Linear Regression | 57.7122    | 61.0832   |

### 17:

17.a: 


Who is the best model?
* Based on the RMSE values in the summary table, the best-performing model on the test set is tabPFN, with the lowest RMSE of 59.86.

Explain shortly the models' results
* XGBoostRegressor: The model performs well on training but significantly worse on test data, indicating overfitting - it learned the training set patterns too well, but doesn't generalize effectively.
 
* CatBoostRegressor: A balanced result - the test RMSE is close to the training RMSE. This suggests good generalization and a solid trade-off between bias and variance. One of the more stable models I got.

* LGBMRegressor: Very low training error but much higher error on the test set - a classic case of overfitting. The model likely memorized the training data and doesn't generalize well.

* RFRegressor: Extremely low training error and a large gap with test error - strong overfitting. While it excels on the training data, it performs poorly on unseen data.

* tabPFNRegressor: Balanced performance. The small gap between train and test RMSE shows that tabPFN generalizes well. Among all models, it has the lowest test RMSE, making it the best performer overall in terms of predictive accuracy.

* This model is very stable, with a small difference between training and test errors. It slightly overfit, but its test RMSE is higher than tabPFN and CatBoost, meaning it is reliable but less accurate.

Explain each of the 4 evaluation metrics.
* R^2 measures how well the model explains the variance in the target variable, it ranges from 0 to 1 where higher values indicate a better fit.
In the context of energy consumption prediction, a high R² means the model successfully captures the patterns in features like temperature and occupancy to explain fluctuations in energy usage. 
* RMSE is measured in the same units as the target variable - in this case, kilowatt-hours (kWh) — making it highly interpretable. It calculates the square root of the average squared difference between predicted and actual values, which means it penalizes larger errors more heavily than smaller ones. This makes RMSE especially useful in my case: when predicting energy usage, a few large mistakes (e.g., underestimating consumption during peak hours) can have operational or financial consequences. Therefore, RMSE is a key metric in this task, as it reflects both average error and sensitivity to outliers.
* MAE is also measured in the same units as the target variable and represents the average absolute difference between the predicted and actual values. Unlike RMSE, MAE treats all errors equally, regardless of their size, making it more robust to outliers. In the context of this project, MAE tells, on average, how many kWh (EnergyConsumption) your predictions deviate from the true values. This is useful when you want a straightforward, reliable estimate of how far off the model tends to be, especially if both small and large errors are equally important to your stakeholders.

* MAPE expresses error as a percentage of the actual values, making it unitless and easy to interpret across different datasets or use cases. For example, a MAPE of 8% means your model is off by 8% on average, or conversely, 92% accurate. This is particularly helpful for communicating model performance to non-technical audiences. In this project, MAPE allows you to quantify how reliable the energy predictions are in relative terms - a key consideration if different buildings or systems have widely varying levels of energy consumption.

17.b:
Related to this context, which is the best suited metric?

* RMSE is the most appropriate and informative metric. Here's why:

RMSE is measured in the same units as the target variable - kilowatt-hours (kWh) - and it penalizes large errors more heavily than smaller ones. This is particularly important in energy forecasting, where a few significant underestimations or overestimations can lead to real-world problems, such as power shortages, overproduction, or inefficient energy distribution. Unlike MAE, which treats all errors equally, RMSE gives more weight to large deviations - and in this context, such deviations can have operational and financial consequences.

Additionally, RMSE is commonly used in energy analytics and engineering applications for its sensitivity to peak error scenarios, making it a natural fit when modeling and optimizing for resource consumption.


